# W_GL_SEGMENT_D SCD Type 2 ETL Process
### Oracle EDW W_GL_SEGMENT_DS to W_GL_SEGMENT_D (Dimension Table - SCD Type 2)

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  -- Check if incremental load is enabled
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  380 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2625027 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS,
  '1900-01-01 00:00:00' AS LOW_DATE,
  '9999-12-31 23:59:59' AS HI_DT,
  COALESCE(
    MAX(date_format(CREATED_ON_DT, 'yyyy-MM-dd HH:mm:ss')),
    '1900-01-01 00:00:00'
  ) AS LAST_ARCHIVE_DATE
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLSEGMENTDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Aggregate source data from W_GL_SEGMENT_DS
CREATE OR REPLACE TEMP VIEW stg_segment_aggregated AS
SELECT
  MAX(X_CUSTOM) AS X_CUSTOM,
  SRC_EFF_TO_DT,
  DATASOURCE_NUM_ID,
  MAX(CREATED_ON_DT) AS CREATED_ON_DT,
  MAX(CHANGED_ON_DT) AS CHANGED_ON_DT,
  MAX(SEGMENT_VAL_CODE) AS SEGMENT_VAL_CODE,
  MAX(AUX3_CHANGED_ON_DT) AS AUX3_CHANGED_ON_DT,
  MAX(AUX1_CHANGED_ON_DT) AS AUX1_CHANGED_ON_DT,
  MAX(AUX4_CHANGED_ON_DT) AS AUX4_CHANGED_ON_DT,
  MAX(TENANT_ID) AS TENANT_ID,
  SRC_EFF_FROM_DT,
  INTEGRATION_ID,
  MAX(CREATED_BY_ID) AS CREATED_BY_ID,
  MAX(DELETE_FLG) AS DELETE_FLG,
  MAX(SET_ID) AS SET_ID,
  MAX(SEGMENT_LOV_ID) AS SEGMENT_LOV_ID,
  MAX(CHANGED_BY_ID) AS CHANGED_BY_ID,
  MAX(AUX2_CHANGED_ON_DT) AS AUX2_CHANGED_ON_DT
FROM workspace.oracle_edw.w_gl_segment_ds
GROUP BY
  SRC_EFF_TO_DT,
  DATASOURCE_NUM_ID,
  SRC_EFF_FROM_DT,
  INTEGRATION_ID;

In [ ]:
%sql
-- 3) LOOKUP: Get CREATED_BY user dimension data
CREATE OR REPLACE TEMP VIEW lkp_user_created AS
SELECT
  DATASOURCE_NUM_ID,
  ROW_WID,
  INTEGRATION_ID,
  EFFECTIVE_TO_DT,
  EFFECTIVE_FROM_DT
FROM workspace.oracle_edw.w_user_d
WHERE DELETE_FLG = 'N';

In [ ]:
%sql
-- 4) LOOKUP: Get CHANGED_BY user dimension data
CREATE OR REPLACE TEMP VIEW lkp_user_changed AS
SELECT
  DATASOURCE_NUM_ID,
  ROW_WID,
  INTEGRATION_ID,
  EFFECTIVE_TO_DT,
  EFFECTIVE_FROM_DT
FROM workspace.oracle_edw.w_user_d
WHERE DELETE_FLG = 'N';

In [ ]:
%sql
-- 5) FLOW TABLE (I$): Build flow table with transformations and lookups
CREATE OR REPLACE TEMP VIEW flow_table AS
SELECT
  0 AS SCD1_WID,
  s.SEGMENT_LOV_ID,
  COALESCE(s.SEGMENT_VAL_CODE, '__NOT_APPLICABLE__') AS SEGMENT_VAL_CODE,
  COALESCE(uc.ROW_WID, 0) AS CREATED_BY_WID,
  COALESCE(uch.ROW_WID, 0) AS CHANGED_BY_WID,
  s.CREATED_ON_DT,
  s.CHANGED_ON_DT,
  s.AUX1_CHANGED_ON_DT,
  s.AUX2_CHANGED_ON_DT,
  s.AUX3_CHANGED_ON_DT,
  s.AUX4_CHANGED_ON_DT,
  s.SRC_EFF_FROM_DT,
  s.SRC_EFF_TO_DT,
  CASE WHEN s.DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
  s.DATASOURCE_NUM_ID,
  s.INTEGRATION_ID,
  s.SET_ID,
  s.TENANT_ID,
  s.X_CUSTOM,
  COALESCE(s.SRC_EFF_FROM_DT, to_timestamp(p.LOW_DATE, 'yyyy-MM-dd HH:mm:ss')) AS EFFECTIVE_FROM_DT,
  to_timestamp(p.HI_DT, 'yyyy-MM-dd HH:mm:ss') AS EFFECTIVE_TO_DT,
  'Y' AS IND_CURRENT_FLG,
  'I' AS IND_UPDATE
FROM stg_segment_aggregated s
CROSS JOIN etl_params p
LEFT OUTER JOIN lkp_user_created uc
  ON s.DATASOURCE_NUM_ID = uc.DATASOURCE_NUM_ID
  AND s.CREATED_BY_ID = uc.INTEGRATION_ID
  AND s.CHANGED_ON_DT >= uc.EFFECTIVE_FROM_DT
  AND s.CHANGED_ON_DT < uc.EFFECTIVE_TO_DT
LEFT OUTER JOIN lkp_user_changed uch
  ON s.DATASOURCE_NUM_ID = uch.DATASOURCE_NUM_ID
  AND s.CHANGED_BY_ID = uch.INTEGRATION_ID
  AND s.CHANGED_ON_DT >= uch.EFFECTIVE_FROM_DT
  AND s.CHANGED_ON_DT < uch.EFFECTIVE_TO_DT
WHERE NOT EXISTS (
  SELECT 1
  FROM workspace.oracle_edw.w_gl_segment_d t
  WHERE t.SRC_EFF_FROM_DT = s.SRC_EFF_FROM_DT
    AND t.DATASOURCE_NUM_ID = s.DATASOURCE_NUM_ID
    AND t.INTEGRATION_ID = s.INTEGRATION_ID
    AND COALESCE(t.CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.AUX1_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.AUX1_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.AUX2_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.AUX2_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.AUX3_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.AUX3_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd'))
    AND COALESCE(t.AUX4_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.AUX4_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd'))
);

In [ ]:
%sql
-- 6) FLAG UPDATES: Identify existing records that need updates
CREATE OR REPLACE TEMP VIEW flow_with_update_flag AS
SELECT
  f.*,
  CASE
    WHEN t.INTEGRATION_ID IS NOT NULL THEN 'U'
    ELSE f.IND_UPDATE
  END AS IND_UPDATE_FINAL,
  CASE
    WHEN t.EFFECTIVE_FROM_DT IS NOT NULL THEN t.EFFECTIVE_FROM_DT
    ELSE f.EFFECTIVE_FROM_DT
  END AS EFFECTIVE_FROM_DT_FINAL
FROM flow_table f
LEFT JOIN (
  SELECT
    SRC_EFF_FROM_DT,
    DATASOURCE_NUM_ID,
    INTEGRATION_ID,
    MAX(EFFECTIVE_FROM_DT) AS EFFECTIVE_FROM_DT
  FROM workspace.oracle_edw.w_gl_segment_d
  GROUP BY SRC_EFF_FROM_DT, DATASOURCE_NUM_ID, INTEGRATION_ID
) t
  ON f.SRC_EFF_FROM_DT = t.SRC_EFF_FROM_DT
  AND f.DATASOURCE_NUM_ID = t.DATASOURCE_NUM_ID
  AND f.INTEGRATION_ID = t.INTEGRATION_ID;

In [ ]:
%sql
-- 7) INSERT SCD1 (Type 1) Records: Create new Type 1 dimension records
INSERT INTO workspace.oracle_edw.w_gl_segment_t1_d
(
  SCD1_WID,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  W_INSERT_DT,
  W_UPDATE_DT,
  ETL_PROC_WID
)
SELECT
  COALESCE(MAX(scd1.SCD1_WID), 0) + ROW_NUMBER() OVER (ORDER BY i.DATASOURCE_NUM_ID, i.INTEGRATION_ID) AS SCD1_WID,
  i.DATASOURCE_NUM_ID,
  i.INTEGRATION_ID,
  current_timestamp() AS W_INSERT_DT,
  current_timestamp() AS W_UPDATE_DT,
  p.ETL_PROC_WID
FROM (
  SELECT DISTINCT
    DATASOURCE_NUM_ID,
    INTEGRATION_ID
  FROM flow_with_update_flag
  WHERE IND_UPDATE_FINAL != 'X'
) i
CROSS JOIN etl_params p
CROSS JOIN (SELECT COALESCE(MAX(SCD1_WID), 0) AS SCD1_WID FROM workspace.oracle_edw.w_gl_segment_t1_d) scd1
LEFT OUTER JOIN workspace.oracle_edw.w_gl_segment_t1_d t1
  ON t1.DATASOURCE_NUM_ID = i.DATASOURCE_NUM_ID
  AND t1.INTEGRATION_ID = i.INTEGRATION_ID
WHERE t1.SCD1_WID IS NULL;

In [ ]:
%sql
-- 8) SOFT DELETE PREPROCESSING: Identify deleted records from PE table
CREATE OR REPLACE TEMP VIEW delete_candidates AS
SELECT
  t.SRC_EFF_FROM_DT,
  t.DATASOURCE_NUM_ID,
  t.INTEGRATION_ID
FROM workspace.oracle_edw.w_gl_segment_d t
LEFT OUTER JOIN workspace.oracle_edw.w_gl_segment_d_pe s
  ON t.SRC_EFF_FROM_DT = s.SRC_EFF_FROM_DT
  AND t.DATASOURCE_NUM_ID = s.DATASOURCE_NUM_ID
  AND t.INTEGRATION_ID = s.INTEGRATION_ID
CROSS JOIN etl_params p
WHERE s.SRC_EFF_FROM_DT IS NULL
  AND s.DATASOURCE_NUM_ID IS NULL
  AND s.INTEGRATION_ID IS NULL
  AND t.DELETE_FLG = 'N'
  AND t.CREATED_ON_DT > to_timestamp(p.LAST_ARCHIVE_DATE, 'yyyy-MM-dd HH:mm:ss')
  AND t.ROW_WID > 0
  AND EXISTS (
    SELECT 1
    FROM workspace.oracle_edw.w_gl_segment_d_pe ds
    WHERE t.DATASOURCE_NUM_ID = ds.DATASOURCE_NUM_ID
  );

In [ ]:
%sql
-- 9) SOFT DELETE: Clear existing deletes from queue
DELETE FROM workspace.oracle_edw.w_gl_segment_d_del d
WHERE EXISTS (
  SELECT 1
  FROM workspace.oracle_edw.w_gl_segment_d f
  WHERE f.SRC_EFF_FROM_DT = d.SRC_EFF_FROM_DT
    AND f.DATASOURCE_NUM_ID = d.DATASOURCE_NUM_ID
    AND f.INTEGRATION_ID = d.INTEGRATION_ID
    AND f.DELETE_FLG = 'Y'
);

In [ ]:
%sql
-- 10) INSERT DELETE QUEUE: Add records to delete queue from PE comparison
INSERT INTO workspace.oracle_edw.w_gl_segment_d_del
(
  SRC_EFF_FROM_DT,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID
)
SELECT
  SRC_EFF_FROM_DT,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID
FROM delete_candidates;

In [ ]:
%sql
-- 11) INSERT DELETE QUEUE FROM STAGING: Add explicitly deleted records from flow
INSERT INTO workspace.oracle_edw.w_gl_segment_d_del
(
  SRC_EFF_FROM_DT,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID
)
SELECT
  t.SRC_EFF_FROM_DT,
  t.DATASOURCE_NUM_ID,
  t.INTEGRATION_ID
FROM flow_with_update_flag t
WHERE t.DELETE_FLG = 'Y'
  AND NOT EXISTS (
        SELECT 1
        FROM workspace.oracle_edw.w_gl_segment_d_del del
        WHERE t.SRC_EFF_FROM_DT = del.SRC_EFF_FROM_DT
          AND t.DATASOURCE_NUM_ID = del.DATASOURCE_NUM_ID
          AND t.INTEGRATION_ID = del.INTEGRATION_ID
      );

In [ ]:
%sql
-- 12) FIX EFFECTIVE DATES (Incremental): Update effective dates for existing records
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING (
  SELECT
        t0.ROW_WID,
        COALESCE(
          MIN(t1.EFFECTIVE_FROM_DT),
          to_timestamp((SELECT HI_DT FROM etl_params), 'yyyy-MM-dd HH:mm:ss')
        ) AS NEW_EFFECTIVE_TO_DT,
        CASE WHEN MIN(t1.EFFECTIVE_FROM_DT) IS NULL THEN 'Y' ELSE 'N' END AS NEW_CURRENT_FLG
      FROM workspace.oracle_edw.w_gl_segment_d t0
      INNER JOIN workspace.oracle_edw.w_gl_segment_d i0
        ON t0.DATASOURCE_NUM_ID = i0.DATASOURCE_NUM_ID
        AND t0.INTEGRATION_ID = i0.INTEGRATION_ID
        AND t0.EFFECTIVE_FROM_DT <= i0.EFFECTIVE_FROM_DT
        AND t0.EFFECTIVE_TO_DT >= i0.EFFECTIVE_FROM_DT
        AND t0.DELETE_FLG = 'N'
      CROSS JOIN etl_params p
      LEFT JOIN workspace.oracle_edw.w_gl_segment_d t1
        ON t0.DATASOURCE_NUM_ID = t1.DATASOURCE_NUM_ID
        AND t0.INTEGRATION_ID = t1.INTEGRATION_ID
        AND t0.EFFECTIVE_FROM_DT < t1.EFFECTIVE_FROM_DT
        AND t1.DELETE_FLG = 'N'
      WHERE i0.ETL_PROC_WID = p.ETL_PROC_WID
      GROUP BY t0.ROW_WID
    ) src
    ON tgt.ROW_WID = src.ROW_WID
    WHEN MATCHED THEN
      UPDATE SET
        tgt.EFFECTIVE_TO_DT = src.NEW_EFFECTIVE_TO_DT,
        tgt.CURRENT_FLG = src.NEW_CURRENT_FLG;

In [ ]:
%sql
-- 13) SOFT DELETE ON TARGET: Mark records in delete queue as deleted
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING workspace.oracle_edw.w_gl_segment_d_del AS src
ON tgt.SRC_EFF_FROM_DT = src.SRC_EFF_FROM_DT
  AND tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
WHEN MATCHED THEN
  UPDATE SET
    tgt.DELETE_FLG = 'Y',
    tgt.CURRENT_FLG = 'N',
    tgt.W_UPDATE_DT = current_timestamp(),
    tgt.ETL_PROC_WID = (SELECT ETL_PROC_WID FROM etl_params);

In [ ]:
%sql
-- 14) INSERT NEW DIMENSION RECORDS: Add new SCD Type 2 records
INSERT INTO workspace.oracle_edw.w_gl_segment_d
(
  ROW_WID,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  SEGMENT_LOV_ID,
  SEGMENT_VAL_CODE,
  SET_ID,
  TENANT_ID,
  X_CUSTOM,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  W_INSERT_DT,
  W_UPDATE_DT,
  ETL_PROC_WID,
  SCD1_WID,
  CURRENT_FLG,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT
)
SELECT
  COALESCE(MAX(d.ROW_WID), 0) + ROW_NUMBER() OVER (
    ORDER BY s.DATASOURCE_NUM_ID, s.INTEGRATION_ID, s.EFFECTIVE_FROM_DT_FINAL
  ) AS ROW_WID,
  s.DATASOURCE_NUM_ID,
  s.INTEGRATION_ID,
  s.SEGMENT_LOV_ID,
  s.SEGMENT_VAL_CODE,
  s.SET_ID,
  s.TENANT_ID,
  s.X_CUSTOM,
  s.CREATED_BY_WID,
  s.CHANGED_BY_WID,
  s.CREATED_ON_DT,
  s.CHANGED_ON_DT,
  s.AUX1_CHANGED_ON_DT,
  s.AUX2_CHANGED_ON_DT,
  s.AUX3_CHANGED_ON_DT,
  s.AUX4_CHANGED_ON_DT,
  s.SRC_EFF_FROM_DT,
  s.SRC_EFF_TO_DT,
  s.DELETE_FLG,
  current_timestamp() AS W_INSERT_DT,
  current_timestamp() AS W_UPDATE_DT,
  p.ETL_PROC_WID,
  scd1.SCD1_WID,
  s.IND_CURRENT_FLG AS CURRENT_FLG,
  s.EFFECTIVE_FROM_DT_FINAL AS EFFECTIVE_FROM_DT,
  s.EFFECTIVE_TO_DT
FROM flow_with_update_flag s
CROSS JOIN etl_params p
CROSS JOIN (SELECT COALESCE(MAX(ROW_WID), 0) AS ROW_WID FROM workspace.oracle_edw.w_gl_segment_d) d
INNER JOIN workspace.oracle_edw.w_gl_segment_t1_d scd1
  ON scd1.DATASOURCE_NUM_ID = s.DATASOURCE_NUM_ID
  AND scd1.INTEGRATION_ID = s.INTEGRATION_ID
WHERE s.IND_UPDATE_FINAL IN ('I', 'S');

In [ ]:
%sql
-- 15) UPDATE EXISTING ROWS: Update Type 1 changes to existing dimension records
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING (
  SELECT
        s.*,
        p.ETL_PROC_WID
      FROM flow_with_update_flag s
      CROSS JOIN etl_params p
      WHERE s.IND_UPDATE_FINAL = 'U'
    ) src
    ON tgt.EFFECTIVE_FROM_DT = src.EFFECTIVE_FROM_DT_FINAL
      AND tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
      AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
    WHEN MATCHED THEN
      UPDATE SET
        tgt.SEGMENT_LOV_ID = src.SEGMENT_LOV_ID,
        tgt.SEGMENT_VAL_CODE = src.SEGMENT_VAL_CODE,
        tgt.CREATED_BY_WID = src.CREATED_BY_WID,
        tgt.CHANGED_BY_WID = src.CHANGED_BY_WID,
        tgt.CREATED_ON_DT = src.CREATED_ON_DT,
        tgt.CHANGED_ON_DT = src.CHANGED_ON_DT,
        tgt.AUX1_CHANGED_ON_DT = src.AUX1_CHANGED_ON_DT,
        tgt.AUX2_CHANGED_ON_DT = src.AUX2_CHANGED_ON_DT,
        tgt.AUX3_CHANGED_ON_DT = src.AUX3_CHANGED_ON_DT,
        tgt.AUX4_CHANGED_ON_DT = src.AUX4_CHANGED_ON_DT,
        tgt.SRC_EFF_FROM_DT = src.SRC_EFF_FROM_DT,
        tgt.SRC_EFF_TO_DT = src.SRC_EFF_TO_DT,
        tgt.DELETE_FLG = src.DELETE_FLG,
        tgt.SET_ID = src.SET_ID,
        tgt.TENANT_ID = src.TENANT_ID,
        tgt.X_CUSTOM = src.X_CUSTOM,
        tgt.W_UPDATE_DT = current_timestamp(),
        tgt.ETL_PROC_WID = src.ETL_PROC_WID;

In [ ]:
%sql
-- 16) HISTORIZE OLD ROWS: Update existing records to make room for new inserts
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING (
  SELECT
        t.ROW_WID,
        MIN(s.EFFECTIVE_FROM_DT_FINAL) AS NEW_EFFECTIVE_TO_DT
      FROM workspace.oracle_edw.w_gl_segment_d t
      INNER JOIN flow_with_update_flag s
        ON s.DATASOURCE_NUM_ID = t.DATASOURCE_NUM_ID
        AND s.INTEGRATION_ID = t.INTEGRATION_ID
        AND s.EFFECTIVE_FROM_DT_FINAL > t.EFFECTIVE_FROM_DT
        AND s.EFFECTIVE_FROM_DT_FINAL < t.EFFECTIVE_TO_DT
        AND s.IND_UPDATE_FINAL IN ('I', 'S')
      WHERE t.DELETE_FLG = 'N'
      GROUP BY t.ROW_WID
    ) src
    ON tgt.ROW_WID = src.ROW_WID
    WHEN MATCHED THEN
      UPDATE SET
        tgt.CURRENT_FLG = 'N',
        tgt.EFFECTIVE_TO_DT = src.NEW_EFFECTIVE_TO_DT;

In [ ]:
%sql
-- 17) UPDATE CONTROL TABLE: Upsert W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT
        p.DATASOURCE_NUM_ID AS datasource_num_id,
        'SILOS_SIL_GLSEGMENTDIMENSION' AS package_name,
        'W_GL_SEGMENT_D' AS target_table_name,
        p.ETL_USAGE_CODE AS etl_usage_code,
        p.ETL_PROC_WID AS etl_proc_wid,
        p.EXECUTION_ID AS load_plan_id,
        current_timestamp() AS etl_load_date,
        CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
        date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
      FROM etl_params p
    ) src
    ON tgt.datasource_num_id = src.datasource_num_id
      AND tgt.package_name = src.package_name
      AND tgt.etl_usage_code = src.etl_usage_code
    WHEN MATCHED THEN
      UPDATE SET
        tgt.target_table_name = src.target_table_name,
        tgt.etl_proc_wid = src.etl_proc_wid,
        tgt.load_plan_id = src.load_plan_id,
        tgt.wip_load_start_date = src.wip_load_start_date,
        tgt.etl_load_date = src.etl_load_date,
        tgt.committed = src.committed
    WHEN NOT MATCHED THEN
      INSERT (
        datasource_num_id, package_name, target_table_name, etl_usage_code,
        etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
      )
      VALUES (
        src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
        src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed
      );

In [ ]:
%sql
-- 18) INSERT HISTORY LOG: Record ETL execution in history table
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
        p.DATASOURCE_NUM_ID,
        'SILOS_SIL_GLSEGMENTDIMENSION',
        'W_GL_SEGMENT_D',
        p.ETL_USAGE_CODE,
        p.ETL_PROC_WID,
        p.EXECUTION_ID,
        p.ETL_PROC_WID AS SESSION_ID,
        date_add(current_timestamp(), -p.PRUNE_DAYS) AS WIP_LOAD_START_DATE,
        NULL AS LAST_MAX_DATE,
        current_timestamp() AS ETL_LOAD_DATE,
        CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS COMMITTED
      FROM etl_params p;

In [ ]:
%sql
-- 19) CLEANUP: Drop temporary views
DROP VIEW IF EXISTS flow_with_update_flag;
DROP VIEW IF EXISTS delete_candidates;
DROP VIEW IF EXISTS flow_table;
DROP VIEW IF EXISTS lkp_user_changed;
DROP VIEW IF EXISTS lkp_user_created;
DROP VIEW IF EXISTS stg_segment_aggregated;
DROP VIEW IF EXISTS etl_params;

---
## ETL Process Summary

### Source → Target
- **Source**: `workspace.oracle_edw.w_gl_segment_ds` (Staging)
- **Target**: `workspace.oracle_edw.w_gl_segment_d` (Dimension - SCD Type 2)

### Key Features
1. **SCD Type 2**: Maintains historical records with effective dating
2. **Change Detection**: Tracks changes via multiple timestamp columns
3. **Soft Delete**: Uses DELETE_FLG instead of physical deletes
4. **User Lookups**: Resolves CREATED_BY and CHANGED_BY to W_USER_D
5. **Type 1 Support**: Separate W_GL_SEGMENT_T1_D table for Type 1 attributes
6. **Delete Detection**: Compares with W_GL_SEGMENT_D_PE for deleted records

### Tables Used
- **Main Tables**: W_GL_SEGMENT_DS (source), W_GL_SEGMENT_D (target)
- **Support Tables**: W_GL_SEGMENT_T1_D (Type 1), W_GL_SEGMENT_D_DEL (delete queue), W_GL_SEGMENT_D_PE (primary extract)
- **Lookup Tables**: W_USER_D (user dimension)
- **Control Tables**: W_ETL_LOAD_DATES, W_ETL_LOAD_DATES_LOG

### Change Detection Logic
Records are compared on:
- CHANGED_ON_DT
- AUX1_CHANGED_ON_DT
- AUX2_CHANGED_ON_DT
- AUX3_CHANGED_ON_DT
- AUX4_CHANGED_ON_DT

### Notes
- ROW_WID is auto-generated using MAX + ROW_NUMBER pattern
- SCD1_WID is generated similarly for Type 1 records
- Effective dates use '1900-01-01' as LOW_DATE and '9999-12-31' as HI_DT
- Incremental load enabled by default (checks W_ETL_LOAD_DATES)
---